## Este caderno aplica o algoritmo Apriori para construir regras de associação e identificar quais são os conjuntos de disciplinas que geralmente são ministradas em conjunto pelo mesmo professor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder


%matplotlib inline

In [2]:
# É preciso mudar o nome do arquivo para mudar a região analisada
# Por default esse caderno analisa a região Centro-Oeste

# Lê os dados da região selecionada usando o Dask para não sobrecarregar a memória
dc = dd.read_csv('DADOS/DOCENTES_CO.CSV', sep='|', dtype='object')

In [3]:
# Listar as colunas que não precisamos para remover do Dataframe Dask, antes de computar
# As colunas necessárias para identificar quais disciplinas são ministradas en comjunto são
# as disciplinas e as colunas 'TP_TIPO_DOCENTE', 'TP_ETAPA_ENSINO', 'TP_DEPENDENCIA', 'TP_LOCALIZACAO'

v_cols = ['NU_ANO_CENSO',
 'CO_PESSOA_FISICA',
 'NU_DIA',
 'NU_MES',
 'NU_ANO',
 'NU_IDADE_REFERENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'CO_PAIS_ORIGEM',
 'CO_UF_NASC',
 'CO_MUNICIPIO_NASC',
 'CO_UF_END',
 'CO_MUNICIPIO_END',
 'IN_NECESSIDADE_ESPECIAL',
 'IN_CEGUEIRA',
 'IN_BAIXA_VISAO',
 'IN_SURDEZ',
 'IN_DEF_AUDITIVA',
 'IN_SURDOCEGUEIRA',
 'IN_DEF_FISICA',
 'IN_DEF_INTELECTUAL',
 'IN_DEF_MULTIPLA',
 'TP_ESCOLARIDADE',
 'TP_NORMAL_MAGISTERIO',
 'TP_SITUACAO_CURSO_1',
 'CO_AREA_CURSO_1',
 'CO_CURSO_1',
 'IN_LICENCIATURA_1',
 'IN_COM_PEDAGOGICA_1',
 'NU_ANO_INICIO_1',
 'NU_ANO_CONCLUSAO_1',
 'TP_TIPO_IES_1',
 'CO_IES_1',
 'TP_SITUACAO_CURSO_2',
 'CO_AREA_CURSO_2',
 'CO_CURSO_2',
 'IN_LICENCIATURA_2',
 'IN_COM_PEDAGOGICA_2',
 'NU_ANO_INICIO_2',
 'NU_ANO_CONCLUSAO_2',
 'TP_TIPO_IES_2',
 'CO_IES_2',
 'TP_SITUACAO_CURSO_3',
 'CO_AREA_CURSO_3',
 'CO_CURSO_3',
 'IN_LICENCIATURA_3',
 'IN_COM_PEDAGOGICA_3',
 'NU_ANO_INICIO_3',
 'NU_ANO_CONCLUSAO_3',
 'TP_TIPO_IES_3',
 'CO_IES_3',
 'IN_ESPECIALIZACAO',
 'IN_MESTRADO',
 'IN_DOUTORADO',
 'IN_POS_NENHUM',
 'IN_ESPECIFICO_CRECHE',
 'IN_ESPECIFICO_PRE_ESCOLA',
 'IN_ESPECIFICO_ANOS_INICIAIS',
 'IN_ESPECIFICO_ANOS_FINAIS',
 'IN_ESPECIFICO_ENS_MEDIO',
 'IN_ESPECIFICO_EJA',
 'IN_ESPECIFICO_ED_ESPECIAL',
 'IN_ESPECIFICO_ED_INDIGENA',
 'IN_ESPECIFICO_CAMPO',
 'IN_ESPECIFICO_AMBIENTAL',
 'IN_ESPECIFICO_DIR_HUMANOS',
 'IN_ESPECIFICO_DIV_SEXUAL',
 'IN_ESPECIFICO_DIR_ADOLESC',
 'IN_ESPECIFICO_AFRO',
 'IN_ESPECIFICO_OUTROS',
 'IN_ESPECIFICO_NENHUM',
 'TP_TIPO_CONTRATACAO',
 'ID_TURMA',
 'TP_TIPO_TURMA',
 'TP_MEDIACAO_DIDATICO_PEDAGO',
 'IN_ESPECIAL_EXCLUSIVA',
 'IN_REGULAR',
 'IN_EJA',
 'IN_PROFISSIONALIZANTE',
 'CO_CURSO_EDUC_PROFISSIONAL',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_ENTIDADE',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_ZONA_RESIDENCIAL',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_CONVENIADA_PP',
 'TP_CONVENIO_PODER_PUBLICO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'TP_REGULAMENTACAO',
 'TP_LOCALIZACAO_DIFERENCIADA',
 'IN_EDUCACAO_INDIGENA']


# Remove as colunas listadas e computar para carregar os dados
dc_co = dc.drop(v_cols, axis=1).compute()

# Imprime a lista de colunas que foram carregadas
dc_co.columns.tolist()


['IN_DISC_QUIMICA',
 'IN_DISC_FISICA',
 'IN_DISC_MATEMATICA',
 'IN_DISC_BIOLOGIA',
 'IN_DISC_CIENCIAS',
 'IN_DISC_LINGUA_PORTUGUESA',
 'IN_DISC_LINGUA_INGLES',
 'IN_DISC_LINGUA_ESPANHOL',
 'IN_DISC_LINGUA_FRANCES',
 'IN_DISC_LINGUA_OUTRA',
 'IN_DISC_LINGUA_INDIGENA',
 'IN_DISC_ARTES',
 'IN_DISC_EDUCACAO_FISICA',
 'IN_DISC_HISTORIA',
 'IN_DISC_GEOGRAFIA',
 'IN_DISC_FILOSOFIA',
 'IN_DISC_ENSINO_RELIGIOSO',
 'IN_DISC_ESTUDOS_SOCIAIS',
 'IN_DISC_SOCIOLOGIA',
 'IN_DISC_EST_SOCIAIS_SOCIOLOGIA',
 'IN_DISC_INFORMATICA_COMPUTACAO',
 'IN_DISC_PROFISSIONALIZANTE',
 'IN_DISC_ATENDIMENTO_ESPECIAIS',
 'IN_DISC_DIVER_SOCIO_CULTURAL',
 'IN_DISC_LIBRAS',
 'IN_DISC_PEDAGOGICAS',
 'IN_DISC_OUTRAS',
 'TP_TIPO_DOCENTE',
 'TP_ETAPA_ENSINO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO']

In [4]:
#Preenche as células NaN com 0 (zero)
dc_co = dc_co.fillna(0)

In [5]:
#converte os Dados de object para Numeric
dc_co = dc_co.convert_objects(convert_numeric=True)

/home/zunokan/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [6]:
#lst é a lista que contém os identificadores do Ensino Médio
lst=[25,26,27,28,29,30,31,32,33,34,35,36,37,38]
dc_co['TP_ETAPA_ENSINO'] = dc_co['TP_ETAPA_ENSINO'].astype(float)

#elimina todas as linhas de docentes que não atuam no Ensino Médio
dc_co = dc_co[dc_co.TP_ETAPA_ENSINO.isin(lst)]

In [7]:
#lst contém os professores que estão atuando em sala de aula
lst=[1]
dc_co['TP_TIPO_DOCENTE'] = dc_co['TP_TIPO_DOCENTE'].astype(float)

#Elimina as linhas dos professores que nao estão atuando em sala de aula
dc_co = dc_co[dc_co.TP_TIPO_DOCENTE.isin(lst)]

In [8]:
#lst contém as dependências estaduais e municipais
lst=[2,3]
dc_co['TP_DEPENDENCIA'] = dc_co['TP_DEPENDENCIA'].astype(float)
dc_co = dc_co[dc_co.TP_DEPENDENCIA.isin(lst)]

In [9]:
#lst contém os docentes da zona urbana
lst=[1]
dc_co['TP_LOCALIZACAO'] = dc_co['TP_LOCALIZACAO'].astype(float)
urbana = dc_co[dc_co.TP_LOCALIZACAO.isin(lst)]


#lst contém os docentes da zona rural
lst=[2]
dc_co['TP_LOCALIZACAO'] = dc_co['TP_LOCALIZACAO'].astype(float)
rural = dc_co[dc_co.TP_LOCALIZACAO.isin(lst)]


In [10]:
# Após a filtragem por etapa e dependencia da escola, deixar apenas as informações das disciplinas
# Também é preciso remover a coluna IN_DISC_EST_SOCIAIS_SOCIOLOGIA pois ela é redundante 
u_cols=[27,28,29,30]

# Elimina as colunas citadas acima
dc_co = dc_co.drop(dc_co.columns[u_cols], axis=1)
urbana = urbana.drop(urbana.columns[u_cols], axis=1)
rural = rural.drop(rural.columns[u_cols], axis=1)

# Lista 
dc_co.columns.tolist()

['IN_DISC_QUIMICA',
 'IN_DISC_FISICA',
 'IN_DISC_MATEMATICA',
 'IN_DISC_BIOLOGIA',
 'IN_DISC_CIENCIAS',
 'IN_DISC_LINGUA_PORTUGUESA',
 'IN_DISC_LINGUA_INGLES',
 'IN_DISC_LINGUA_ESPANHOL',
 'IN_DISC_LINGUA_FRANCES',
 'IN_DISC_LINGUA_OUTRA',
 'IN_DISC_LINGUA_INDIGENA',
 'IN_DISC_ARTES',
 'IN_DISC_EDUCACAO_FISICA',
 'IN_DISC_HISTORIA',
 'IN_DISC_GEOGRAFIA',
 'IN_DISC_FILOSOFIA',
 'IN_DISC_ENSINO_RELIGIOSO',
 'IN_DISC_ESTUDOS_SOCIAIS',
 'IN_DISC_SOCIOLOGIA',
 'IN_DISC_EST_SOCIAIS_SOCIOLOGIA',
 'IN_DISC_INFORMATICA_COMPUTACAO',
 'IN_DISC_PROFISSIONALIZANTE',
 'IN_DISC_ATENDIMENTO_ESPECIAIS',
 'IN_DISC_DIVER_SOCIO_CULTURAL',
 'IN_DISC_LIBRAS',
 'IN_DISC_PEDAGOGICAS',
 'IN_DISC_OUTRAS']

In [11]:
# Aplicação do algoritmo apriori para as escolas da zona urbana
result = apriori(urbana, min_support=0.01, use_colnames=True)

# Ordena os resultados por ocorrência
result = result.sort_values(by="support", ascending=False)

# Salva o resultado ordenado em um arquivo .csv
result.to_csv('RESULTADOS/RES_CO_URBANA', sep='\t')

In [12]:
# Aplicação do algoritmo apriori para as escolas da zona rural
result = apriori(rural, min_support=0.01, use_colnames=True)

# Ordena os resultados por ocorrência
result = result.sort_values(by="support", ascending=False)

# Salva o resultado ordenado em um arquivo .csv
result.to_csv('RESULTADOS/RES_CO_RURAL', sep='\t')